# People Analytics Case Study

## Libraries

In [1]:
# import libraries
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import getpass

## 1. Importing Data from MySQL

In [2]:
# Connection parameters
bd = "hr_data"
password = "password"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + bd

# Create the connection engine
engine = create_engine(connection_string)

## 2. SQL Queries for Data Analysis

In [3]:
# 1. What is the gender breakdown of employees in the company?
query = """
SELECT gender, COUNT(*) AS count
FROM employees
GROUP BY gender;
"""

# Execute the query and display the result
gender_breakdown = pd.read_sql(query, engine)

# Visualize the result
gender_breakdown

,gender,count
0,Male,10273
1,Female,9361
2,Non-Conforming,549


In [4]:
# 2. What is the race/ethnicity breakdown of employees in the company?
query = """
SELECT race, COUNT(*) AS count
FROM employees
GROUP BY race
ORDER BY count DESC;
"""

# Execute the query and display the result
race_breakdown = pd.read_sql(query, engine)

# Visualize the result
race_breakdown

,race,count
0,White,5755
1,Two or More Races,3309
2,Black or African American,3298
3,Asian,3232
4,Hispanic or Latino,2263
5,American Indian or Alaska Native,1216
6,Native Hawaiian or Other Pacific Islander,1110


In [5]:
# 3. What is the age distribution of employees in the company?

# Get the max and min age
query_age_range = """
SELECT MAX(age) AS max_age, MIN(age) AS min_age
FROM employees;
"""

# Execute the query and store the result in a DataFrame
age_range = pd.read_sql(query_age_range, engine)

# Age distribution by group
query_age_distribution = """
SELECT 
  CASE 
    WHEN age < 30 THEN '20-29'
    WHEN age < 40 THEN '30-39'
    WHEN age < 50 THEN '40-49'
    ELSE '50-59'
  END AS age_group, COUNT(*) AS count
FROM employees
GROUP BY age_group
ORDER BY count DESC;
"""

# Execute the query and store the result in a DataFrame
age_distribution = pd.read_sql(query_age_distribution, engine)

# Visualize the results
age_range, age_distribution

(   max_age  min_age
 0       56       19,
   age_group  count
 0     20-29   5820
 1     30-39   5611
 2     40-49   5370
 3     50-59   3382)

In [6]:
# 4. How many employees work at headquarters versus remote locations?
query = """
SELECT location, COUNT(*) AS count
FROM employees
GROUP BY location;
"""

# Execute the query and store the result in a DataFrame
location_breakdown = pd.read_sql(query, engine)

# Visualize the result
location_breakdown

,location,count
0,Headquarters,15169
1,Remote,5014


In [7]:
# 5. What is the average length of employment for employees who have been terminated?
query = """
SELECT round(avg(DATEDIFF(term_date, hire_date) / 365.25), 0) AS avg_emp_length
FROM employees
WHERE term_date IS NOT NULL;
"""

# Get the average employment length
avg_emp_length_df = pd.read_sql(query, engine)

# Display the result
print(avg_emp_length_df)

   avg_emp_length
0             7.0


In [8]:
# 6. How does the gender distribution vary across departments?
query = """
SELECT department, gender, count(*) AS employees
FROM employees
GROUP BY department, gender
ORDER BY department, employees DESC;
"""

# Execute the query and fetch the results
results = pd.read_sql(query, engine)
results

,department,gender,employees
0,Accounting,Male,1555
1,Accounting,Female,1391
2,Accounting,Non-Conforming,87
3,Auditing,Male,25
4,Auditing,Female,21
5,Business Development,Male,773
6,Business Development,Female,677
7,Business Development,Non-Conforming,45
8,Engineering,Male,3077
9,Engineering,Female,2828


In [9]:
# 7. What is the distribution of job titles across the company?
query = """
SELECT job_title, COUNT(*) AS employees
FROM employees
GROUP BY job_title
ORDER BY employees DESC
LIMIT 10;
"""

# Execute the query and fetch the results
results = pd.read_sql(query, engine)
results

,job_title,employees
0,Research Assistant II,690
1,Business Analyst,635
2,Human Resources Analyst II,545
3,Research Assistant I,478
4,Account Executive,458
5,Data Visualization Specialist,416
6,Staff Accountant I,403
7,Human Resources Analyst,370
8,Software Engineer I,360
9,Systems Administrator I,342


In [10]:
# 8. Which department has the highest turnover rate?
query = """
WITH department_count AS (
    SELECT department, COUNT(*) AS total_count,
           SUM(CASE WHEN term_date IS NOT NULL THEN 1 ELSE 0 END) AS termination_count
    FROM employees
    GROUP BY department
)

SELECT department, 
       ROUND((termination_count / total_count) * 100, 1) AS turnover_rate
FROM department_count
ORDER BY turnover_rate DESC
LIMIT 1;
"""

# Execute the query and fetch the results
results = pd.read_sql(query, engine)
results

,department,turnover_rate
0,Auditing,13.0


In [11]:
# 9. What is the turnover rate across job titles?
query = """
WITH job_title_count AS (
    SELECT job_title, COUNT(*) AS total_count,
           SUM(CASE WHEN term_date IS NOT NULL THEN 1 ELSE 0 END) AS termination_count
    FROM employees
    GROUP BY job_title
)

SELECT job_title, 
       ROUND((termination_count / total_count) * 100, 1) AS turnover_rate
FROM job_title_count
ORDER BY turnover_rate DESC;
"""

# Execute the query and fetch the results
results = pd.read_sql(query, engine)
results

,job_title,turnover_rate
0,Office Assistant II,100.0
1,Executive Secretary,100.0
2,Statistician III,50.0
3,Sales Representative,40.0
4,Engineer II,33.3
...,...,...
179,Account Coordinator,0.0
180,Associate Professor,0.0
181,VP of Training and Development,0.0
182,Office Assistant IV,0.0


In [16]:
# 10. How have turnover rates changed each year based on Term Date?

# This query calculates the turnover rate based on Term Date without filtering based on hire_date,
# and includes data starting from 2003, as the dataset begins in that year.
# It counts the number of employees who left the company in a given year (termination_count)
# and compares it to the total number of employees who were still active in that year (total_count).

query = """
WITH year_cte AS (
    SELECT YEAR(term_date) AS year,
           COUNT(*) AS termination_count,
           (SELECT COUNT(*)
            FROM employees
            WHERE hire_date <= DATE(CONCAT(YEAR(term_date), '-12-31')) 
              AND (term_date IS NULL OR term_date >= DATE(CONCAT(YEAR(term_date), '-01-01')))
           ) AS total_count
    FROM employees
    WHERE term_date IS NOT NULL
    GROUP BY YEAR(term_date)
)

SELECT year, 
       ROUND((termination_count / total_count) * 100, 1) AS turnover_rate
FROM year_cte
ORDER BY year ASC;

"""

# Execute the query and fetch the results
results = pd.read_sql(query, engine)

# Display the results
results

# Explanation:
# - This query calculates the number of terminated employees per year (termination_count) 
#   and the total number of employees who were active or terminated in that year (total_count).
# - Employees are considered active if they were hired before the end of the year and 
#   did not leave before the year in question.
# - The turnover rate is calculated by dividing termination_count by total_count, and 
#   multiplying by 100 to get a percentage.

,year,turnover_rate
0,2004,0.4
1,2005,0.5
2,2006,1.6
3,2007,1.5
4,2008,2.6
5,2009,3.6
6,2010,2.6
7,2011,4.6
8,2012,3.9
9,2013,6.1


In [13]:
# 11. What is the distribution of employees across states?
query = """
SELECT location_state, COUNT(*) AS employees
FROM employees
GROUP BY location_state
ORDER BY employees DESC, location_state;
"""

# Execute the query and fetch the results
results = pd.read_sql(query, engine)
results

,location_state,employees
0,California,16355
1,Oregon,1010
2,Nevada,801
3,Arizona,632
4,Utah,616
5,New Mexico,418
6,Idaho,351


## 3. Exporting Data for Tableau Visualization

In [14]:
# SQL query to execute
consulta = "SELECT * FROM hr_data.employees"

# Read the SQL query into a DataFrame
df_sql = pd.read_sql(consulta, con=engine)

# Save the DataFrame to a CSV file
df_sql.to_csv('../data/cleaned/hr_data_cleaned.csv', index=False)

# Display the first rows of the DataFrame
df_sql.sort_values(by="term_date")

,emp_id,first_name,last_name,birth_date,age,gender,race,department,job_title,location,hire_date,term_date,location_state
13033,64-6403747,Briggs,Eastway,1971-03-13,53,Female,American Indian or Alaska Native,Business Development,Research Assistant II,Headquarters,2003-12-03,2004-04-15,California
17625,87-3003781,Menard,Bril,1992-09-25,32,Female,American Indian or Alaska Native,Services,Service Manager,Headquarters,2004-08-28,2004-10-06,California
8036,39-8448824,Keir,Dechelle,2005-07-22,19,Male,White,Sales,Relationshiop Manager,Headquarters,2004-05-31,2004-10-24,California
6253,31-0270355,Joey,Agney,1980-03-08,44,Female,Black or African American,Support,Support Staff,Headquarters,2004-09-22,2004-11-11,California
3691,18-3127889,Faydra,Matushevich,1999-12-29,25,Male,Two or More Races,Human Resources,Human Resources Analyst,Headquarters,2004-04-29,2004-11-30,California
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20177,99-9610988,Hillie,Renbold,1968-10-30,56,Female,Two or More Races,Engineering,Web Developer II,Headquarters,2004-12-03,NaT,California
20178,99-9797418,Dorella,Garvan,2001-07-08,23,Female,Hispanic or Latino,Research and Development,Research Assistant I,Headquarters,2015-02-08,NaT,California
20179,99-9869877,Dasie,Thorsby,2004-04-19,20,Female,Two or More Races,Services,Service Manager,Headquarters,2020-10-06,NaT,California
20180,99-9919822,Nerty,Wilding,1973-02-09,51,Female,Two or More Races,Training,Junior Trainer,Headquarters,2004-02-08,NaT,California
